In [ ]:

!pip install transformers datasets nltk -q

# ✅ Step 2: Import libraries
import pandas as pd
import torch
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# ✅ Step 3: Load dataset
df = pd.read_csv("/content/all_english_labeled_news (1).csv")
df=df.iloc[32000:38000]
df = df.dropna(subset=["text"]).reset_index(drop=True)
#df = df.reset_index(drop=True)

# ✅ Step 4: Load model and tokenizer (facebook/bart-large-mnli)
model_name = "facebook/bart-large-mnli"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# ✅ Step 5: Truncate to 512 tokens
def truncate_to_512_tokens(text):
    encoded = tokenizer.encode(text, max_length=512, truncation=True)
    return tokenizer.decode(encoded, skip_special_tokens=True)

df["text"] = df["text"].apply(lambda x: truncate_to_512_tokens(str(x)))
df = df[df["text"].str.len() > 30].reset_index(drop=True)

# ✅ Step 6: Moral foundations and hypothesis builder
labels = [
    "Care", "Harm", "Fairness", "Cheating",
    "Loyalty", "Betrayal", "Authority", "Subversion",
    "Sanctity", "Degradation", "Liberty", "Oppression"
]

def make_hypotheses(label_list):
    return [f"This text expresses {label}." for label in label_list]

# ✅ Step 7: Batched classification with progress and timing
def classify_batch_manual(batch_texts, batch_size=16):
    all_results = []
    total = len(batch_texts)
    start_time = time.time()

    for i in tqdm(range(0, total, batch_size)):
        texts = batch_texts[i:i+batch_size]
        batch_out = []

        for text in texts:
            hypotheses = make_hypotheses(labels)
            premise_list = [text] * len(hypotheses)
            inputs = tokenizer(premise_list, hypotheses, padding=True, truncation=True, return_tensors="pt").to(device)

            with torch.no_grad():
                logits = model(**inputs).logits
            probs = softmax(logits, dim=1)[:, 2]  # entailment

            label_scores = [(label, float(score)) for label, score in zip(labels, probs) if score > 0.5]
            batch_out.append(label_scores if label_scores else [("None", 1.0)])

        all_results.extend(batch_out)

        # ⏱ ETA
        elapsed = time.time() - start_time
        avg_time = elapsed / (i + batch_size)
        remaining = avg_time * (total - (i + batch_size))
        print(f"\n⏳ ETA: {remaining/60:.1f} min remaining\n")

    return all_results

# ✅ Step 8: Run classification
results = classify_batch_manual(df["Details"].tolist(), batch_size=16)

# ✅ Step 9: Store results
df["moral_foundations"] = [[label for label, score in r] for r in results]
df["moral_scores"] = [dict(r) for r in results]

# ✅ Step 10: Save to CSV
output_path = "/content/drive/MyDrive/moral_foundation_labeled_bart.csv"
df.to_csv(output_path, index=False)
print(f"✅ Done! File saved to: {output_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

100%|█████████▉| 374/375 [2:36:25<00:26, 26.65s/it]


⏳ ETA: 0.4 min remaining



100%|██████████| 375/375 [2:36:47<00:00, 25.09s/it]


⏳ ETA: 0.0 min remaining



✅ Done! File saved to: /content/drive/MyDrive/moral_foundation_labeled_bart.csv


In [ ]:
merged_df

,Unnamed: 0,Date,Title,Link,Details,newspaper_name,text,sentiment,emotion,top_2_moral_foundations,top_2_moral_scores
0,1,2024-09-24,Ex-IGP Mamun on 4-day remand in murder case,https://www.thedailystar.net/news/bangladesh/c...,A Dhaka court today placed former inspector ge...,The Daily Star,Ex-IGP Mamun on 4-day remand in murder case. A...,LABEL_0,anger,"[Authority, Care]","{'Authority': 0.7845588326454163, 'Care': 0.64..."
1,2,2024-09-24,"Touhid, Jaishankar meet at UNGA, discuss mutua...",https://www.thedailystar.net/news/bangladesh/d...,"Foreign Affairs Adviser, HE Md. Touhid Hossai...",The Daily Star,"Touhid, Jaishankar meet at UNGA, discuss mutua...",LABEL_1,fear,"[Authority, Loyalty]","{'Authority': 0.8064534068107605, 'Loyalty': 0..."
2,3,2024-09-24,Army chief pledges support for Yunus' interim ...,https://www.thedailystar.net/news/bangladesh/n...,Bangladesh's army chief vowed to back the coun...,The Daily Star,Army chief pledges support for Yunus' interim ...,LABEL_1,anger,"[Subversion, Oppression]","{'Subversion': 0.9299159646034241, 'Oppression..."
3,4,2024-09-24,Released top criminals at it again,https://www.thedailystar.net/news/bangladesh/c...,"Nasir Biswas, a 26-year-old mason, was walking...",The Daily Star,Released top criminals at it again. Nasir Bisw...,LABEL_1,anger,"[Sanctity, Subversion]","{'Sanctity': 0.8243871927261353, 'Subversion':..."
4,5,2024-09-24,Shibir now out into the open at DU,https://www.thedailystar.net/campus/news/shibi...,"After around three and a half decades, Islami ...",The Daily Star,Shibir now out into the open at DU. After arou...,LABEL_1,disgust,"[Sanctity, Cheating]","{'Sanctity': 0.8725903630256653, 'Cheating': 0..."
...,...,...,...,...,...,...,...,...,...,...,...
37559,139339,2024-10-05,India foreign minister to visit Pakistan in ra...,https://en.prothomalo.com/international/india/...,India's foreign minister will be in Pakistan l...,Prothom Alo,India foreign minister to visit Pakistan in ra...,LABEL_1,neutral,"[Authority, Care]","{'Authority': 0.7862706184387207, 'Care': 0.57..."
37560,139340,2024-10-05,Initiative to reform Shilpakala Academy Act,https://en.prothomalo.com/entertainment/tkx9v6...,An initiative has been taken to reform Banglad...,Prothom Alo,Initiative to reform Shilpakala Academy Act. A...,LABEL_1,neutral,"[Authority, Care]","{'Authority': 0.8890504837036133, 'Care': 0.70..."
37561,139341,2024-10-05,India bemoan 'harsh' T20 World Cup run-out row,https://en.prothomalo.com/sports/cricket/xzyc3...,India star Jemimah Rodrigues described a run-o...,Prothom Alo,India bemoan 'harsh' T20 World Cup run-out row...,LABEL_1,disgust,"[Harm, Care]","{'Harm': 0.9209035634994507, 'Care': 0.8041914..."
37562,139342,2024-10-05,Reform commissions likely to start operations ...,https://en.prothomalo.com/bangladesh/op5bwbj4iz,Although the notification regarding the five o...,Prothom Alo,Reform commissions likely to start operations ...,LABEL_1,neutral,"[Authority, Loyalty]","{'Authority': 0.800456702709198, 'Loyalty': 0...."


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/merged_framing_dataset.csv')
df

,Unnamed: 0,Date,Title,Link,Details,newspaper_name,text,sentiment,emotion,top_2_moral_foundations,top_2_moral_scores,parsed_morals,clean_text,bertopic_label,time_window,moral_list
0,0,2024-09-24,Ex-IGP Mamun on 4-day remand in murder case,https://www.thedailystar.net/news/bangladesh/c...,A Dhaka court today placed former inspector ge...,The Daily Star,Ex-IGP Mamun on 4-day remand in murder case. A...,label_0,anger,"['Authority', 'Care']","{'Authority': 0.7845588326454163, 'Care': 0.64...","['authority', 'care']",exigp mamun on day remand in murder case a dha...,11,After,"['authority', 'care']"
1,1,2024-09-24,"Touhid, Jaishankar meet at UNGA, discuss mutua...",https://www.thedailystar.net/news/bangladesh/d...,"Foreign Affairs Adviser, HE Md. Touhid Hossai...",The Daily Star,"Touhid, Jaishankar meet at UNGA, discuss mutua...",label_1,fear,"['Authority', 'Loyalty']","{'Authority': 0.8064534068107605, 'Loyalty': 0...","['authority', 'loyalty']",touhid jaishankar meet at unga discuss mutual ...,50,After,"['authority', 'loyalty']"
2,2,2024-09-24,Army chief pledges support for Yunus' interim ...,https://www.thedailystar.net/news/bangladesh/n...,Bangladesh's army chief vowed to back the coun...,The Daily Star,Army chief pledges support for Yunus' interim ...,label_1,anger,"['Subversion', 'Oppression']","{'Subversion': 0.9299159646034241, 'Oppression...","['subversion', 'oppression']",army chief pledges support for yunus interim g...,81,After,"['subversion', 'oppression']"
3,3,2024-09-24,Released top criminals at it again,https://www.thedailystar.net/news/bangladesh/c...,"Nasir Biswas, a 26-year-old mason, was walking...",The Daily Star,Released top criminals at it again. Nasir Bisw...,label_1,anger,"['Sanctity', 'Subversion']","{'Sanctity': 0.8243871927261353, 'Subversion':...","['sanctity', 'subversion']",released top criminals at it again nasir biswa...,-1,After,"['sanctity', 'subversion']"
4,4,2024-09-24,Shibir now out into the open at DU,https://www.thedailystar.net/campus/news/shibi...,"After around three and a half decades, Islami ...",The Daily Star,Shibir now out into the open at DU. After arou...,label_1,disgust,"['Sanctity', 'Cheating']","{'Sanctity': 0.8725903630256653, 'Cheating': 0...","['sanctity', 'cheating']",shibir now out into the open at du after aroun...,-1,After,"['sanctity', 'cheating']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37528,50302,2024-10-05,India foreign minister to visit Pakistan in ra...,https://en.prothomalo.com/international/india/...,India's foreign minister will be in Pakistan l...,Prothom Alo,India foreign minister to visit Pakistan in ra...,label_1,neutral,"['Authority', 'Care']","{'Authority': 0.7862706184387207, 'Care': 0.57...","['authority', 'care']",india foreign minister to visit pakistan in ra...,31,After,"['authority', 'care']"
37529,50303,2024-10-05,Initiative to reform Shilpakala Academy Act,https://en.prothomalo.com/entertainment/tkx9v6...,An initiative has been taken to reform Banglad...,Prothom Alo,Initiative to reform Shilpakala Academy Act. A...,label_1,neutral,"['Authority', 'Care']","{'Authority': 0.8890504837036133, 'Care': 0.70...","['authority', 'care']",initiative to reform shilpakala academy act an...,-1,After,"['authority', 'care']"
37530,50304,2024-10-05,India bemoan 'harsh' T20 World Cup run-out row,https://en.prothomalo.com/sports/cricket/xzyc3...,India star Jemimah Rodrigues described a run-o...,Prothom Alo,India bemoan 'harsh' T20 World Cup run-out row...,label_1,disgust,"['Harm', 'Care']","{'Harm': 0.9209035634994507, 'Care': 0.8041914...","['harm', 'care']",india bemoan harsh t world cup runout row indi...,25,After,"['harm', 'care']"
37531,50305,2024-10-05,Reform commissions likely to start operations ...,https://en.prothomalo.com/bangladesh/op5bwbj4iz,Although the notification regarding the five o...,Prothom Alo,Reform commissions likely to start operations ...,label_1,neutral,"['Authority', 'Loyalty']","{'Authority': 0.800456702709198, '

In [ ]:
# ================================================================
# 1. Install & import the libraries you need
# ================================================================
!pip install --quiet plotly>=5.20 kaleido # kaleido lets Plotly export static PNGs

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns
from google.colab import files, drive
from pathlib import Path

# Optional: prettier default Plotly theme
pio.templates.default = "plotly_white"

# ================================================================
# 2. Load the CSV
#    ──► OPTION A: upload directly in Colab (fastest for a single file)
# ================================================================
# Uncomment if you want to upload manually
# uploaded = files.upload()         # triggers a file‑picker
# df = pd.read_csv(next(iter(uploaded)))  # grabs the first uploaded file

# ================================================================
#    ──► OPTION B: read from Google Drive (better for many files)
# ================================================================
# drive.mount('/content/drive')
# DATA_PATH = Path("/content/drive/MyDrive/research/Sheikh Hasina_role_shift.csv")
# df = pd.read_csv(DATA_PATH)

# If you already placed the file in the Colab working dir, just do:
df = pd.read_csv("/content/Sheikh Hasina_role_shift.csv")

print("Data loaded! Shape:", df.shape)
display(df.head())

# ================================================================
# 3. Basic data sanity checks
# ================================================================
# Identify the date column automatically (assumes only one datetime‑like column)
date_cols = [c for c in df.columns
             if pd.to_datetime(df[c], errors='coerce').notna().sum() > 0.8 * len(df)]
time_col = date_cols[0] if date_cols else None

if time_col:
    df[time_col] = pd.to_datetime(df[time_col])
    print(f"✅ Using '{time_col}' as the time axis")
else:
    raise ValueError("Couldn’t detect a date column automatically ‑‑ set `time_col` manually!")

# ================================================================
# 4. Tidy vs wide format handling
# ================================================================
role_cols = [c for c in df.columns if c not in [time_col]]
wide_like = df[role_cols].apply(pd.api.types.is_numeric_dtype).all()

if wide_like:
    # Example wide‑to‑long reshape: one column per role  → “tidy”
    tidy_df = (df
               .melt(id_vars=time_col,
                     value_vars=role_cols,
                     var_name="role",
                     value_name="count")
               .sort_values(time_col))
else:
    # Already tidy (columns: time, role, count)
    tidy_df = df.copy()

display(tidy_df.head())

# ================================================================
# 5. Static Matplotlib line plot (count by role over time)
# ================================================================
plt.figure(figsize=(12,6))
for role, sub in tidy_df.groupby('role'):
    plt.plot(sub[time_col], sub['count'], label=role, linewidth=2)

plt.title("Role shift trajectory (static view)")
plt.xlabel("Date")
plt.ylabel("Mention count")
plt.legend()
plt.tight_layout()
plt.show()

# ================================================================
# 6. Interactive Plotly line / area chart
# ================================================================
fig_line = px.line(
    tidy_df,
    x=time_col, y='count', color='role',
    title="Role shift trajectory (interactive)",
    template='plotly_white',
    markers=True
)
fig_line.update_layout(hovermode="x unified")
fig_line.show()

# Optional stacked‑area version
fig_area = px.area(
    tidy_df,
    x=time_col, y='count', color='role',
    title="Role shift trajectory – stacked area",
    groupnorm=None,  # remove to stack to 100 %
)
fig_area.update_layout(hovermode="x unified")
fig_area.show()

# ================================================================
# 7. Heat‑map (role vs. time)
# ================================================================
# Create a pivot table for heat‑map
heat_df = tidy_df.pivot_table(
    index='role',
    columns=time_col,
    values='count',
    fill_value=0
)
plt.figure(figsize=(14,6))
sns.heatmap(heat_df, cmap="YlGnBu", linewidths=.5)
plt.title("Role shift heat‑map")
plt.xlabel("Date")
plt.ylabel("Role")
plt.tight_layout()
plt.show()

# ================================================================
# 8. Export anything you need
# ================================================================
# Example: save interactive HTML for sharing
fig_line.write_html("role_shift_interactive.html")
print("Interactive chart saved ▶ role_shift_interactive.html")


Data loaded! Shape: (2, 7)


,time_window,Accused / Scapegoated,Authority / Leader,Hero / Celebrated Figure,Neutral / Unclear,Reformer / Advocate,Victim / Persecuted
0,After,45.58,33.08,5.97,8.03,0.62,6.72
1,Before,20.07,42.55,6.43,22.75,1.34,6.85


✅ Using 'Accused / Scapegoated' as the time axis


/tmp/ipython-input-1-1787579191.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  if pd.to_datetime(df[c], errors='coerce').notna().sum() > 0.8 * len(df)]


,time_window,Accused / Scapegoated,Authority / Leader,Hero / Celebrated Figure,Neutral / Unclear,Reformer / Advocate,Victim / Persecuted
0,After,1970-01-01 00:00:00.000000045,33.08,5.97,8.03,0.62,6.72
1,Before,1970-01-01 00:00:00.000000020,42.55,6.43,22.75,1.34,6.85


KeyError: 'role'

<Figure size 1200x600 with 0 Axes>

In [ ]:
# ⬇︎ 1. Install Plotly (interactive charting)
!pip install --quiet plotly>=5.20

# ⬇︎ 2. Import libraries
import pandas as pd
import plotly.express as px

# ⬇︎ 3. Load the CSV (adjust the path if needed)
df = pd.read_csv("/content/Dr. Yunus_role_shift.csv")

# ⬇︎ 4. Wide ➜ long reshape
tidy_df = df.melt(
    id_vars="time_window",      # column with 'Before' / 'After'
    var_name="role",            # new column holding each role name
    value_name="percentage"     # values column
)

# ⬇︎ 5. Comparative grouped‑bar plot
fig = px.bar(
    tidy_df,
    x="role",
    y="percentage",
    color="time_window",        # two bars per role: Before vs After
    barmode="group",
    template="plotly_white",
    title="DR. Younus • Role framing before vs after 5 Aug 2024",
    labels={"percentage": "Percentage of mentions", "role": ""}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [ ]:
# ======================================================================
#  MASTER ANALYSIS SCRIPT  –  Bangladeshi Newspaper Moral & Sentiment
#  Paste this entire block into ONE Colab cell and press ▶ Run.
#  --------------------------------------------------------------------
#  Author: ChatGPT (OpenAI)         Date: 13 Jul 2025 (Asia/Dhaka)
# ======================================================================

# --------------------------- 0. SETTINGS ------------------------------
USE_DRIVE = True        # ⇦ change to False if not using Google Drive
CSV_PATH  = "/content/drive/MyDrive/merged_framing_dataset.csv"  # ⇦ change me"  # ⇦ change me
CUTOFF    = "2024-08-06"   # regime‑change date (YYYY‑MM‑DD)

# --------------------------- 1. INSTALLS ------------------------------
import subprocess, sys, textwrap, warnings, importlib
pkgs = ["pandas","numpy","scipy","statsmodels","matplotlib","seaborn","tqdm"]
for p in pkgs:
    try:
        importlib.import_module(p)
    except ImportError:
        subprocess.check_call([sys.executable,"-m","pip","install","-q",p])

# --------------------------- 2. DRIVE MOUNT ---------------------------
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

# --------------------------- 3. IMPORTS -------------------------------
import pandas as pd, numpy as np, re, matplotlib.pyplot as plt, seaborn as sns
from datetime import datetime, timezone
from scipy import stats
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore", category=FutureWarning)
sns.set_theme(style="whitegrid")

# --------------------------- 4. LOAD DATA -----------------------------
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df):,} rows.", "Columns:", list(df.columns), sep="\n")

# --------------------------- 5. COLUMN SANITY -------------------------
expected = [
    'Date','Title','Link','Details','newspaper_name','text',
    'sentiment','emotion','top_2_moral_foundations','top_2_moral_scores',
    'parsed_morals','clean_text','bertopic_label','time_window',
    'moral_list','Month'
]
missing = [c for c in expected if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

# --------------------------- 6. FEATURE ENGINEERING -------------------
# 6.1 Datetime → Asia/Dhaka
df['Date'] = (pd.to_datetime(df['Date'], errors='coerce', utc=True)
                .dt.tz_convert('Asia/Dhaka'))
if df['Date'].isna().any():
    raise ValueError("Date parsing failed for some rows.")

# 6.2 Regime flag
cutoff = pd.Timestamp(CUTOFF, tz='Asia/Dhaka')
df['Regime'] = np.where(df['Date'] < cutoff, 'AL', 'Yunus')

# 6.3 Numerical sentiment
sent_map = {'Negative':-1, 'Neutral':0, 'Positive':1}
df['sentiment_score'] = df['sentiment'].replace(sent_map).astype(float)

# 6.4 Dominant moral foundation
df['dominant_moral'] = (df['top_2_moral_foundations']
                        .str.split('[,;/|]')
                        .str[0]
                        .str.strip()
                        .str.title())

# 6.5 Token count
df['tokens'] = df['clean_text'].str.split().str.len()

# 6.6 Auto‑classify ideology
def ideology_from_group(g, thresh=0.10):
    m_al    = g.loc[g.Regime=='AL',    'sentiment_score'].mean()
    m_yunus = g.loc[g.Regime=='Yunus', 'sentiment_score'].mean()
    diff    = m_al - m_yunus
    if diff >  thresh: return 'Pro‑AL'
    if diff < -thresh: return 'Pro‑Yunus'
    return 'Neutral'

df['ideology'] = (df.groupby('newspaper_name', group_keys=False)
                    .apply(ideology_from_group)
                    .reindex(df['newspaper_name']).values)
print("\nIdeology counts:\n", df['ideology'].value_counts())

# --------------------------- 7. STATISTICAL TESTS ---------------------
print("\n" + "="*60 + "\nSTATISTICAL TESTS\n" + "="*60)

# 7.1 Welch t‑test – sentiment
t_stat, p_t = stats.ttest_ind(
    df.loc[df.Regime=='AL',    'sentiment_score'].dropna(),
    df.loc[df.Regime=='Yunus', 'sentiment_score'].dropna(),
    equal_var=False
)
print(f"\nWelch t‑test (Sentiment AL vs Yunus): t = {t_stat:.3f}, p = {p_t:.4g}")

# 7.2 χ² – emotion
emotion_ct = pd.crosstab(df['emotion'], df['Regime'])
chi2_e, p_e, _, _ = stats.chi2_contingency(emotion_ct)
print(f"\nChi‑square (Emotion × Regime): χ² = {chi2_e:.2f}, p = {p_e:.4g}")

# 7.3 χ² – morals
moral_ct = pd.crosstab(df['dominant_moral'], df['Regime'])
chi2_m, p_m, _, _ = stats.chi2_contingency(moral_ct)
print(f"\nChi‑square (Moral × Regime): χ² = {chi2_m:.2f}, p = {p_m:.4g}")

# 7.3b Multinomial logit – morals
mnlogit = smf.mnlogit('dominant_moral ~ Regime', data=df).fit(disp=0)
print("\nMultinomial logit (dominant_moral ~ Regime) – truncated:\n",
      textwrap.indent(mnlogit.summary().as_text()[:1100], "  "), "...")

# 7.4 Two‑way ANOVA – sentiment ~ Regime × ideology
model = smf.ols('sentiment_score ~ C(Regime)*C(ideology)', data=df).fit()
print("\nTwo‑way ANOVA – sentiment:\n", anova_lm(model, typ=2))

# 7.5 Topic × Regime χ²
topic_ct = pd.crosstab(df['bertopic_label'], df['Regime'])
chi2_t, p_t2, _, _ = stats.chi2_contingency(topic_ct)
print(f"\nChi‑square (Topic × Regime): χ² = {chi2_t:.2f}, p = {p_t2:.4g}")

# 7.5b Per‑topic logistic – quick screen
topic_results = []
for topic in tqdm(df['bertopic_label'].unique(), desc="Topic logits"):
    tmp = df.assign(is_topic=(df['bertopic_label']==topic).astype(int))
    res = smf.logit('is_topic ~ Regime', data=tmp).fit(disp=0)
    topic_results.append({
        'topic': topic,
        'coef(Regime_Yunus)': res.params.get('Regime[T.Yunus]', np.nan),
        'p': res.pvalues.get('Regime[T.Yunus]', np.nan)
    })
topic_df = (pd.DataFrame(topic_results)
            .sort_values('p')
            .reset_index(drop=True))
print("\nTop 10 topics most affected by regime:\n", topic_df.head(10))

# --------------------------- 8. FIGURES ------------------------------
print("\nRendering figures ...")
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

# 8.1 Sentiment timeline
sns.lineplot(ax=axs[0], data=df,
             x='Date', y='sentiment_score', hue='Regime',
             estimator='mean', ci='sd')
axs[0].set_title('Mean Sentiment (±SD)')

# 8.2 Emotion stacked count
emotion_ct.plot(kind='bar', stacked=True, ax=axs[1])
axs[1].set_title('Emotion Counts by Regime')
axs[1].set_ylabel('Articles')

# 8.3 Moral share
(moral_ct.div(moral_ct.sum(0), axis=1).T
         .plot(kind='bar', stacked=True, ax=axs[2]))
axs[2].set_title('Dominant Moral Foundation Share')
plt.tight_layout()
plt.show()

print("\n✔ DONE – all stats & plots complete.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 37,533 rows.
Columns:
['Unnamed: 0', 'Date', 'Title', 'Link', 'Details', 'newspaper_name', 'text', 'sentiment', 'emotion', 'top_2_moral_foundations', 'top_2_moral_scores', 'parsed_morals', 'clean_text', 'bertopic_label', 'time_window', 'moral_list']


ValueError: Missing columns: ['Month']

In [ ]:
df

,Unnamed: 0,Date,Title,Link,Details,newspaper_name,text,sentiment,emotion,top_2_moral_foundations,top_2_moral_scores,parsed_morals,clean_text,bertopic_label,time_window,moral_list
0,0,2024-09-24,Ex-IGP Mamun on 4-day remand in murder case,https://www.thedailystar.net/news/bangladesh/c...,A Dhaka court today placed former inspector ge...,The Daily Star,Ex-IGP Mamun on 4-day remand in murder case. A...,label_0,anger,"['Authority', 'Care']","{'Authority': 0.7845588326454163, 'Care': 0.64...","['authority', 'care']",exigp mamun on day remand in murder case a dha...,11,After,"['authority', 'care']"
1,1,2024-09-24,"Touhid, Jaishankar meet at UNGA, discuss mutua...",https://www.thedailystar.net/news/bangladesh/d...,"Foreign Affairs Adviser, HE Md. Touhid Hossai...",The Daily Star,"Touhid, Jaishankar meet at UNGA, discuss mutua...",label_1,fear,"['Authority', 'Loyalty']","{'Authority': 0.8064534068107605, 'Loyalty': 0...","['authority', 'loyalty']",touhid jaishankar meet at unga discuss mutual ...,50,After,"['authority', 'loyalty']"
2,2,2024-09-24,Army chief pledges support for Yunus' interim ...,https://www.thedailystar.net/news/bangladesh/n...,Bangladesh's army chief vowed to back the coun...,The Daily Star,Army chief pledges support for Yunus' interim ...,label_1,anger,"['Subversion', 'Oppression']","{'Subversion': 0.9299159646034241, 'Oppression...","['subversion', 'oppression']",army chief pledges support for yunus interim g...,81,After,"['subversion', 'oppression']"
3,3,2024-09-24,Released top criminals at it again,https://www.thedailystar.net/news/bangladesh/c...,"Nasir Biswas, a 26-year-old mason, was walking...",The Daily Star,Released top criminals at it again. Nasir Bisw...,label_1,anger,"['Sanctity', 'Subversion']","{'Sanctity': 0.8243871927261353, 'Subversion':...","['sanctity', 'subversion']",released top criminals at it again nasir biswa...,-1,After,"['sanctity', 'subversion']"
4,4,2024-09-24,Shibir now out into the open at DU,https://www.thedailystar.net/campus/news/shibi...,"After around three and a half decades, Islami ...",The Daily Star,Shibir now out into the open at DU. After arou...,label_1,disgust,"['Sanctity', 'Cheating']","{'Sanctity': 0.8725903630256653, 'Cheating': 0...","['sanctity', 'cheating']",shibir now out into the open at du after aroun...,-1,After,"['sanctity', 'cheating']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37528,50302,2024-10-05,India foreign minister to visit Pakistan in ra...,https://en.prothomalo.com/international/india/...,India's foreign minister will be in Pakistan l...,Prothom Alo,India foreign minister to visit Pakistan in ra...,label_1,neutral,"['Authority', 'Care']","{'Authority': 0.7862706184387207, 'Care': 0.57...","['authority', 'care']",india foreign minister to visit pakistan in ra...,31,After,"['authority', 'care']"
37529,50303,2024-10-05,Initiative to reform Shilpakala Academy Act,https://en.prothomalo.com/entertainment/tkx9v6...,An initiative has been taken to reform Banglad...,Prothom Alo,Initiative to reform Shilpakala Academy Act. A...,label_1,neutral,"['Authority', 'Care']","{'Authority': 0.8890504837036133, 'Care': 0.70...","['authority', 'care']",initiative to reform shilpakala academy act an...,-1,After,"['authority', 'care']"
37530,50304,2024-10-05,India bemoan 'harsh' T20 World Cup run-out row,https://en.prothomalo.com/sports/cricket/xzyc3...,India star Jemimah Rodrigues described a run-o...,Prothom Alo,India bemoan 'harsh' T20 World Cup run-out row...,label_1,disgust,"['Harm', 'Care']","{'Harm': 0.9209035634994507, 'Care': 0.8041914...","['harm', 'care']",india bemoan harsh t world cup runout row indi...,25,After,"['harm', 'care']"
37531,50305,2024-10-05,Reform commissions likely to start operations ...,https://en.prothomalo.com/bangladesh/op5bwbj4iz,Although the notification regarding the five o...,Prothom Alo,Reform commissions likely to start operations ...,label_1,neutral,"['Authority', 'Loyalty']","{'Authority': 0.800456702709198, '